# Build Docker image 

In this notebook, we will build the docker container that contains the ResNet152 model, Flask web application, model driver and all dependencies.
Make sure you have logged in using docker login.

In [1]:
import os
from os import path
import json

In [2]:
!mkdir flaskwebapp
!mkdir flaskwebapp/nginx
!mkdir flaskwebapp/etc

In [4]:
!cp resnet152.py flaskwebapp
!cp driver.py flaskwebapp
!ls flaskwebapp

driver.py  etc	nginx  resnet152.py


Below, we create the module for the Flask web application.

In [5]:
%%writefile flaskwebapp/app.py

from flask import Flask, request
import logging
import json
import driver

app = Flask(__name__)
predict_for = driver.get_model_api()
 
@app.route("/score", methods = ['POST'])
def scoreRRS():
    """ Endpoint for scoring
    """
    if request.headers['Content-Type'] != 'application/json':
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    request_input = request.json['input']
    response = predict_for(request_input)
    print(response)
    return json.dumps({'result': str(response)})


@app.route("/")
def healthy():
    return "Healthy"

# Tensorflow Version
@app.route('/version', methods = ['GET'])
def version_request():
    return driver.version()

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

Writing flaskwebapp/app.py


In [6]:
%%writefile flaskwebapp/wsgi.py
from app import app as application

def create():
    print("Initialising")
    application.run(host='127.0.0.1', port=5000)

Writing flaskwebapp/wsgi.py


Here, we write the configuration for the Nginx which creates a proxy between ports **80** and **5000**.

In [7]:
%%writefile flaskwebapp/nginx/app
server {
    listen 80;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

Writing flaskwebapp/nginx/app


In [8]:
%%writefile flaskwebapp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Writing flaskwebapp/gunicorn_logging.conf


In [9]:
%%writefile flaskwebapp/kill_supervisor.py
import sys
import os
import signal

def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()

Writing flaskwebapp/kill_supervisor.py


In [10]:
%%writefile flaskwebapp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true                ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

Writing flaskwebapp/etc/supervisord.conf


We create a custom image based on the CUDA 8 image from NVIDIA and install all the necessary dependencies. This is in order to try and keep the size of the image as small as possible.

In [11]:
%%writefile flaskwebapp/requirements.txt

Pillow==5.0.0
click==6.7
configparser==3.5.0
Flask==0.12.2
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.18.4
h5py==2.6.0

Writing flaskwebapp/requirements.txt


In [12]:
%%writefile flaskwebapp/dockerfile

FROM nvidia/cuda:8.0-cudnn6-devel-ubuntu16.04
    
RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list

USER root
RUN mkdir /code
WORKDIR /code
RUN chmod -R a+w /code
ADD . /code/

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        wget \
        curl \
        git \
        supervisor \
        nginx && \
    rm -rf /var/lib/apt/lists/* &&\
    rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ 

ENV PYTHON_VERSION=3.5
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy pyyaml scipy \
    ipython pandas jupyter ipykernel scikit-learn && \
    /opt/conda/bin/conda clean -ya
    
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
    
RUN pip install --upgrade pip && \
    pip install tensorflow-gpu==1.4.1 && \
    pip install keras==2.1.5 && \
    pip install -r /code/requirements.txt && \       
    /opt/conda/bin/conda clean -yt

EXPOSE 8888
EXPOSE 5000
EXPOSE 80

CMD ["supervisord", "-c", "/code/etc/supervisord.conf"]

Writing flaskwebapp/dockerfile


The image name below referes to our dockerhub account. If you wish to push the image to your account make sure you change this.

In [13]:
image_name = "fboylu/kerastf-gpu"
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

In [14]:
!docker build -t $image_name -f $docker_file_location $application_path --no-cache

Sending build context to Docker daemon  33.79kB
Step 1/17 : FROM nvidia/cuda:8.0-cudnn6-devel-ubuntu16.04
 ---> 547cf50ecba4
Step 2/17 : RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list
 ---> Running in b632df4c1348
 ---> a571d88d503d
Removing intermediate container b632df4c1348
Step 3/17 : USER root
 ---> Running in f72cf6f4e925
 ---> e5ce6fa2e53e
Removing intermediate container f72cf6f4e925
Step 4/17 : RUN mkdir /code
 ---> Running in 773432a04f9b
 ---> 891bbce15961
Removing intermediate container 773432a04f9b
Step 5/17 : WORKDIR /code
 ---> 08c799153060
Removing intermediate container 3151a6351203
Step 6/17 : RUN chmod -R a+w /code
 ---> Running in eecf5b8200ea
 ---> 75d60e78232f
Removing intermediate container eecf5b8200ea
Step 7/17 : ADD . /code/
 ---> 5ec888c3f0a9
Step 8/17 : RUN apt-get update && apt-get install -y --no-install-recommends         build-essential         ca-certificates

Get:15 http://archive.ubuntu.com/ubuntu xenial/main amd64 libffi6 amd64 3.2.1-4 [17.8 kB]
Get:16 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libp11-kit0 amd64 0.23.2-5~ubuntu16.04.1 [105 kB]
Get:17 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libtasn1-6 amd64 4.7-3ubuntu0.16.04.3 [43.5 kB]
Get:18 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libgnutls30 amd64 3.4.10-4ubuntu1.4 [548 kB]
Get:19 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libroken18-heimdal amd64 1.7~git20150920+dfsg-4ubuntu1.16.04.1 [41.4 kB]
Get:20 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libasn1-8-heimdal amd64 1.7~git20150920+dfsg-4ubuntu1.16.04.1 [174 kB]
Get:21 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libhcrypto4-heimdal amd64 1.7~git20150920+dfsg-4ubuntu1.16.04.1 [85.0 kB]
Get:22 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libheimbase1-heimdal amd64 1.7~git20150920+dfsg-4ubuntu1.16.04.1 [29.3 kB]
Get:23 http:

Unpacking libxml2:amd64 (2.9.3+dfsg1-1ubuntu0.5) ...
Selecting previously unselected package libarchive13:amd64.
Preparing to unpack .../libarchive13_3.1.2-11ubuntu0.16.04.3_amd64.deb ...
Unpacking libarchive13:amd64 (3.1.2-11ubuntu0.16.04.3) ...
Selecting previously unselected package libkrb5support0:amd64.
Preparing to unpack .../libkrb5support0_1.13.2+dfsg-5ubuntu2_amd64.deb ...
Unpacking libkrb5support0:amd64 (1.13.2+dfsg-5ubuntu2) ...
Selecting previously unselected package libk5crypto3:amd64.
Preparing to unpack .../libk5crypto3_1.13.2+dfsg-5ubuntu2_amd64.deb ...
Unpacking libk5crypto3:amd64 (1.13.2+dfsg-5ubuntu2) ...
Selecting previously unselected package libkeyutils1:amd64.
Preparing to unpack .../libkeyutils1_1.5.9-8ubuntu1_amd64.deb ...
Unpacking libkeyutils1:amd64 (1.5.9-8ubuntu1) ...
Selecting previously unselected package libkrb5-3:amd64.
Preparing to unpack .../libkrb5-3_1.13.2+dfsg-5ubuntu2_amd64.deb ...
Unpacking libkrb5-3:amd64 (1.13.2+dfsg-5ubuntu2) ...
Selecting pre

Selecting previously unselected package libjbig0:amd64.
Preparing to unpack .../libjbig0_2.1-3.1_amd64.deb ...
Unpacking libjbig0:amd64 (2.1-3.1) ...
Selecting previously unselected package libpng12-0:amd64.
Preparing to unpack .../libpng12-0_1.2.54-1ubuntu1_amd64.deb ...
Unpacking libpng12-0:amd64 (1.2.54-1ubuntu1) ...
Selecting previously unselected package ucf.
Preparing to unpack .../archives/ucf_3.0036_all.deb ...
Moving old data out of the way
Unpacking ucf (3.0036) ...
Selecting previously unselected package openssl.
Preparing to unpack .../openssl_1.0.2g-1ubuntu4.11_amd64.deb ...
Unpacking openssl (1.0.2g-1ubuntu4.11) ...
Selecting previously unselected package ca-certificates.
Preparing to unpack .../ca-certificates_20170717~16.04.1_all.deb ...
Unpacking ca-certificates (20170717~16.04.1) ...
Selecting previously unselected package libcurl3-gnutls:amd64.
Preparing to unpack .../libcurl3-gnutls_7.47.0-1ubuntu2.7_amd64.deb ...
Unpacking libcurl3-gnutls:amd64 (7.47.0-1ubuntu2.7) 

debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
Setting up openssl (1.0.2g-1ubuntu4.11) ...
Setting up ca-certificates (20170717~16.04.1) ...
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
Setting up libcurl3-gnutls:amd64 (7.47.0-1ubuntu2.7) ...
Setting up libgeoip1:amd64 (1.6.9-1) ...
Setting up libxdmcp6:amd64 (1:1.1.2-1.1) ...
Setting up libxcb1:amd64 (1.11.1-1ubuntu1) ...
Setting up libx11-data (2:1.6.3-1ubuntu2) ...
Setting up libx11-6:amd64 (2:1.6.3-1ubuntu2) ...
Setting up wget (1.17.1-1ubuntu1.3) ...
Setting up curl (7.47.0-1ubuntu2.7) ...
Setting up fonts-dejavu-core (2.35-1) ...
Setting up fontconfig-config (2.11.94-0ubuntu1.1) ...
Setting up liberror-perl (0.17-1.2) ...
Setting up git-man (1:2.7.4-0ubuntu1.3) ...
Setting up git (1:2.7.4-0ubuntu1.3) ...
Setting

ptyprocess 0.5.2: ########## | 100% 
backcall 0.1.0: ########## | 100% 
dbus 1.13.2: ########## | 100% 
python-dateutil 2.7.2: ########## | 100% 
gstreamer 1.12.4: ########## | 100% 
pyyaml 3.12: ########## | 100% 
libstdcxx-ng 7.2.0: ########## | 100% 
traitlets 4.3.2: ########## | 100% 
ca-certificates 2018.03.07: ########## | 100% 
ipykernel 4.8.2: ########## | 100% 
libgfortran-ng 7.2.0: ########## | 100% 
jupyter_client 5.2.3: ########## | 100% 
chardet 3.0.4: ########## | 100% 
urllib3 1.22: ########## | 100% 
openssl 1.0.2o: ########## | 100% 
pandas 0.22.0: ########## | 100% 
pip 9.0.3: ########## | 100% 
jinja2 2.10: ########## | 100% 
webencodings 0.5.1: ########## | 100% 
ipywidgets 7.2.0: ########## | 100% 
mkl 2018.0.2: ########## | 100% 
freetype 2.8: ########## | 100% 
pexpect 4.4.0: ########## | 100% 
ipython 6.3.1: ########## | 100% 
terminado 0.8.1: ########## | 100% 
testpath 0.3.1: ########## | 100% 
bleach 2.1.3: ########## | 100% 
idna 2.6: ########## | 100% 
nbfo

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
#
# To activate this environment, use:
# > source activate py3.5
#
# To deactivate an active environment, use:
# > source deactivate
#


Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
wcwidth-0.1.7-py35hcd08066_0.tar.bz2          25 KB
qtconsole-4.3.1-py35h4626a06_0.tar.bz2       151 KB
parso-0.1.1-py35h1b200a3_0.tar.bz2           118 KB
terminado-0.8.1-py35_1.tar.bz2                21 KB
nbconvert-5.3.1-py35hc5194e3_0.tar.bz2       397 KB
pysocks-1.6.7-py36hd97a5b1_1.tar.bz2          22 KB
idna-2.6-py35h8605a33_1.tar.bz2              123 KB
jupyter_client-5.2.3-py35_0.tar.bz2          125 KB
pip-9.0.3-py35_0.tar.bz2                     2.3 MB
prompt_toolkit-1.0.15-py35hc09de7a_0.tar.bz2     343 KB
conda-4.4.10-py36_0.tar.bz2                  929 KB
chardet-3.0.4-py36h0f667ec_1.tar.bz2         190 KB
send2tra

Removed openssl-1.0.2o-h20670df_0.tar.bz2
Removed zlib-1.2.11-ha838bed_2.tar.bz2
Removed python-dateutil-2.7.2-py35_0.tar.bz2
Removed icu-58.2-h9c2bf20_1.tar.bz2
Removed sphinx-1.7.2-py35_0.tar.bz2
Removed jsonschema-2.6.0-py35h4395190_0.tar.bz2
Removed freetype-2.8-hab7d2ae_1.tar.bz2
Removed sip-4.19.8-py35hf484d3e_0.tar.bz2
Removed jpeg-9b-h024ee3a_2.tar.bz2
Removed jupyter-1.0.0-py35_4.tar.bz2
Removed jedi-0.11.1-py35_1.tar.bz2
Removed bleach-2.1.3-py35_0.tar.bz2
Removed pandas-0.22.0-py35hf484d3e_0.tar.bz2
Removed pyopenssl-17.5.0-py36h20ba746_0.tar.bz2
Removed typing-3.6.4-py35_0.tar.bz2
Removed asn1crypto-0.24.0-py35_0.tar.bz2
Removed numpy-1.14.2-py35hdbf6ddf_1.tar.bz2
Removed mistune-0.8.3-py35_0.tar.bz2
Removed tk-8.6.7-hc745277_3.tar.bz2
Removed imagesize-1.0.0-py35_0.tar.bz2
Removed xz-5.2.3-h55aa19d_2.tar.bz2
Removed libpng-1.6.34-hb9fc6fc_0.tar.bz2
Removed libxml2-2.9.8-hf84eae3_0.tar.bz2
Removed pcre-8.41-hc27e229_1.tar.bz2
Removed pycparser-2.18-py35h61b3040_1.tar.bz2
Re

  Running setup.py bdist_wheel for html5lib: started
  Running setup.py bdist_wheel for html5lib: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/6f/85/6c/56b8e1292c6214c4eb73b9dda50f53e8e977bf65989373c962
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 2.1.3
    Uninstalling bleach-2.1.3:
      Successfully uninstalled bleach-2.1.3
  Running setup.py bdist_wheel for configparser: started
  Running setup.py bdist_wheel for configparser: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/1c/bd/b4/277af3f6c40645661b4cd1c21df26aca0f2e1e9714a1d4cda8
  Running setup.py bdist_wheel for json-logging-py: started
  Running setup.py bdist_wheel for json-logging-py: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/3c/57/3d/085046797b926eb8b2da12a89ad4a423a4afc3ab8569e85e2f
  Runni

In [15]:
!docker push $image_name

The push refers to a repository [docker.io/fboylu/kerastf-gpu]

4414f4e8: Preparing 
8c750024: Preparing 
39973c9c: Preparing 
a1e6033a: Preparing 
a7ee05bb: Preparing 
4818dc36: Preparing 
00bfcccf: Preparing 
6006b2f8: Preparing 
27c508b0: Preparing 
e48572eb: Preparing 
748c63e7: Preparing 
b03ecbb7: Preparing 
b2bd3356: Preparing 
4c622b50: Preparing 
ea2bb533: Preparing 
89ea437e: Preparing 
8b9b1b5b: Preparing 


c750024: Pushing  1.094GB/2.14GBBPushing  402.8MB/887.5MB

c750024: Pushed   2.216GB/2.14GBlatest: digest: sha256:906e3af08eb9ff43778f0837b71debbcb7a4c5a349f9af0595c464c45897648b size: 4086


In [16]:
print('Docker image name {}'.format(image_name))

Docker image name fboylu/kerastf-gpu


We can now [test our image locally](03_TestLocally.ipynb).